In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

# Question 5: Sales ROI: What is the average profit expectation from upgrades and sales after mortgage payments? (10 points)

In [2]:
# Load the CSV files into variables
file_paths = {
    "growth": "./Metro_zhvf_growth_uc_sfrcondo_tier_0.33_0.67_month.csv",
    "value": "./Metro_zhvi_uc_sfr_tier_0.33_0.67_sm_sa_month.csv",
    "median_price": "./Metro_median_sale_price_uc_sfrcondo_month.csv",
    "list_price": "./Metro_mlp_uc_sfrcondo_month.csv",
    "sold_above_list": "./Metro_pct_sold_above_list_uc_sfrcondo_month.csv",
    "sold_below_list": "./Metro_pct_sold_below_list_uc_sfrcondo_month.csv"
}

# Loaded CSV variables into pandas DataFrames
dataframes = {name: pd.read_csv(path) for name, path in file_paths.items()}

# Display only the first 5 rows of each DataFrame using head()
dataframes_heads = {name: df.head() for name, df in dataframes.items()}

# Output the first 5 rows for each dataset
for name, head in dataframes_heads.items():
    print(f"First 5 rows of {name} dataset:")
    print(head)
    print("\n")

First 5 rows of growth dataset:
   RegionID  SizeRank       RegionName RegionType StateName    BaseDate  \
0    394913         1     New York, NY        msa        NY  10/31/2024   
1    753899         2  Los Angeles, CA        msa        CA  10/31/2024   
2    394463         3      Chicago, IL        msa        IL  10/31/2024   
3    394514         4       Dallas, TX        msa        TX  10/31/2024   
4    394692         5      Houston, TX        msa        TX  10/31/2024   

   11/30/2024  1/31/2025  10/31/2025  
0        -0.5       -0.8         1.1  
1        -0.7       -0.8         2.2  
2        -1.0       -2.2         0.7  
3        -1.1       -2.1         1.3  
4        -0.9       -1.7         0.8  


First 5 rows of value dataset:
   RegionID  SizeRank       RegionName RegionType StateName    1/31/2023  \
0    394913         1     New York, NY        msa        NY  618315.3511   
1    753899         2  Los Angeles, CA        msa        CA  903986.6773   
2    394463         3 

# Observations
- All data shares identifiers "RegionID", "RegionName", "RegionType", and "StateName"
- The time-series and formatting of the data is consistent across all the datasets.
- growth: Percentage change in values, representing property value growth or decline.
- value: Median home values over time.
- median_price: Median sale prices over time.
- list_price: Median list prices over time.
- sold_above_list and sold_below_list: Proportion of homes sold above or below the list price over time.

## Merging the datasets into one using "RegionID" as the temporal alignment.
### Furthering the data by creating derivations:
- Upgrade Costs: Based on growth or a percentage of median_price.
- Sales Revenue: Derived from median_price.
- Mortgage Payments: Using assumptions like average loan terms and interest rates.

In [3]:
# Merge datasets on RegionID, RegionName, and StateName
key_columns = ["RegionID", "RegionName", "StateName"]

# Reduce to time-series data for simplicity (assuming alignment by RegionID)
merged_data = dataframes["value"][key_columns].copy()

# Merge other datasets
for name, df in dataframes.items():
    time_columns = [col for col in df.columns if col not in key_columns]
    df_reduced = df[key_columns + time_columns]
    merged_data = pd.merge(merged_data, df_reduced, on=key_columns, suffixes=('', f'_{name}'))

# Inspect the merged structure
merged_data.head(), merged_data.columns.tolist()

(   RegionID       RegionName StateName  SizeRank RegionType    BaseDate  \
 0    394913     New York, NY        NY         1        msa  10/31/2024   
 1    753899  Los Angeles, CA        CA         2        msa  10/31/2024   
 2    394463      Chicago, IL        IL         3        msa  10/31/2024   
 3    394514       Dallas, TX        TX         4        msa  10/31/2024   
 4    394692      Houston, TX        TX         5        msa  10/31/2024   
 
    11/30/2024  1/31/2025  10/31/2025  SizeRank_value  ...  \
 0        -0.5       -0.8         1.1               1  ...   
 1        -0.7       -0.8         2.2               2  ...   
 2        -1.0       -2.2         0.7               3  ...   
 3        -1.1       -2.1         1.3               4  ...   
 4        -0.9       -1.7         0.8               5  ...   
 
   12/31/2023_sold_below_list  1/31/2024_sold_below_list  \
 0                   0.406383                   0.428314   
 1                   0.426111                   

# Calculate and analyze ROI:

### Explanation:
- Sales Revenue: The income generated from property sales.
- Upgrade Costs: Expenses incurred for property upgrades.
- Mortgage Payments: The total mortgage payments over the analysis period.
### Steps:
1. Calculate Upgrade Costs: Determine Sales Revenue from median sale prices.
   - Upgrade costs can be derived from the growth percentage and median_price.
     
>$
\text{Upgrade Costs} = \text{Median Price} * \text{Growth Percentage}
$

2. Estimate Revenue: Estimate the Upgrade Costs using growth percentages or as a fraction of median prices.
   - Revenue can be estimated using median_price adjusted by sales percentages <br /> (sold_above_list and sold_below_list).

3. Model Mortgage Payments: Calculate Mortgage Payments based on sale price, loan terms, and interest rates.
   - Formula:
>$
M = P \cdot \frac{r(1 + r)^n}{(1 + r)^n - 1}
$

   - Where:<br />
>M: Monthly payment<br />
P: Loan amount (principal)<br />
r: Monthly interest rate (annual rate ÷ 12)<br />
n: Total number of payments (loan term in months)

4. Compute ROI: Compute the ROI for each region and time period using the formula above.
>$
\text{ROI} = \frac{\text{Sales Revenue} - (\text{Upgrade Costs} + \text{Mortgage Payments})}{\text{Mortgage Payments}}
$

In [5]:
# Inspect column names to identify growth-related columns
print("All columns in merged_data:")
print(merged_data.columns.tolist())

# Adjust growth column selection based on actual column names
# Assuming growth columns correspond to specific date patterns or ranges
growth_columns = [col for col in merged_data.columns if col in ['11/30/2024', '1/31/2025', '10/31/2025']]
print("Identified growth-related columns:", growth_columns)

# Ensure alignment between price and growth columns
aligned_price_columns = [col for col in price_columns if any(date in col for date in growth_columns)]
print("Aligned price columns:", aligned_price_columns)

# Re-attempt calculation if growth columns are identified
if aligned_price_columns and growth_columns:
    upgrade_costs = merged_data[aligned_price_columns].values * merged_data[growth_columns].values
    upgrade_costs_df = pd.DataFrame(
        upgrade_costs,
        columns=[col.replace("median_price", "upgrade_costs") for col in aligned_price_columns],
        index=merged_data.index
    )
    merged_data_with_upgrades = pd.concat([merged_data, upgrade_costs_df], axis=1)
else:
    print("No growth columns found or alignment issue.")

# Inspect the result
merged_data_with_upgrades.head() if 'merged_data_with_upgrades' in locals() else merged_data.head()

All columns in merged_data:
['RegionID', 'RegionName', 'StateName', 'SizeRank', 'RegionType', 'BaseDate', '11/30/2024', '1/31/2025', '10/31/2025', 'SizeRank_value', 'RegionType_value', '1/31/2023', '2/28/2023', '3/31/2023', '4/30/2023', '5/31/2023', '6/30/2023', '7/31/2023', '8/31/2023', '9/30/2023', '10/31/2023', '11/30/2023', '12/31/2023', '1/31/2024', '2/29/2024', '3/31/2024', '4/30/2024', '5/31/2024', '6/30/2024', '7/31/2024', '8/31/2024', '9/30/2024', '10/31/2024', 'SizeRank_median_price', 'RegionType_median_price', '1/31/2023_median_price', '2/28/2023_median_price', '3/31/2023_median_price', '4/30/2023_median_price', '5/31/2023_median_price', '6/30/2023_median_price', '7/31/2023_median_price', '8/31/2023_median_price', '9/30/2023_median_price', '10/31/2023_median_price', '11/30/2023_median_price', '12/31/2023_median_price', '1/31/2024_median_price', '2/29/2024_median_price', '3/31/2024_median_price', '4/30/2024_median_price', '5/31/2024_median_price', '6/30/2024_median_price', '7

,RegionID,RegionName,StateName,SizeRank,RegionType,BaseDate,11/30/2024,1/31/2025,10/31/2025,SizeRank_value,...,12/31/2023_sold_below_list,1/31/2024_sold_below_list,2/29/2024_sold_below_list,3/31/2024_sold_below_list,4/30/2024_sold_below_list,5/31/2024_sold_below_list,6/30/2024_sold_below_list,7/31/2024_sold_below_list,8/31/2024_sold_below_list,9/30/2024_sold_below_list
0,394913,"New York, NY",NY,1,msa,10/31/2024,-0.5,-0.8,1.1,1,...,0.406383,0.428314,0.463802,0.419320,0.375195,0.357391,0.323827,0.331516,0.347761,0.378329
1,753899,"Los Angeles, CA",CA,2,msa,10/31/2024,-0.7,-0.8,2.2,2,...,0.426111,0.460034,0.383346,0.348691,0.325138,0.316547,0.333910,0.353801,0.394207,0.411886
2,394463,"Chicago, IL",IL,3,msa,10/31/2024,-1.0,-2.2,0.7,3,...,0.532863,0.543956,0.509206,0.450741,0.377788,0.366030,0.353733,0.368958,0.417902,0.459814
3,394514,"Dallas, TX",TX,4,msa,10/31/2024,-1.1,-2.1,1.3,4,...,0.613289,0.623790,0.583297,0.517837,0.511471,0.501287,0.535623,0.562001,0.608491,0.619420
4,394692,"Houston, TX",TX,5,msa,10/31/2024,-0.9,-1.7,0.8,5,...,0.676135,0.691283,0.647386,0.621079,0.593125,0.587870,0.590875,0.612032,0.645077,0.647457


In [8]:
# Inspect column patterns for alignment
print("Inspecting column patterns for alignment...\n")

# Group and identify key columns
columns_summary = {
    "Median Price Columns": [col for col in merged_data.columns if "median_price" in col],
    "Sold Above List Columns": [col for col in merged_data.columns if "sold_above_list" in col],
    "Sold Below List Columns": [col for col in merged_data.columns if "sold_below_list" in col],
    "Growth Columns": [col for col in merged_data.columns if col in ['11/30/2024', '1/31/2025', '10/31/2025']],
}

# Print column group summaries
for key, columns in columns_summary.items():
    print(f"{key}: {columns[:5]}{'...' if len(columns) > 5 else ''}")

# Attempt to align and calculate Upgrade Costs based on aligned temporal data
aligned_columns = {
    key: [
        col for col in columns_summary["Median Price Columns"]
        if any(date in col for date in columns_summary["Growth Columns"])
    ]
    for key in ["Median Price Columns"]
}

# Recalculate Upgrade Costs if alignment is successful
if aligned_columns["Median Price Columns"] and columns_summary["Growth Columns"]:
    upgrade_costs = (
        merged_data[aligned_columns["Median Price Columns"]].values *
        merged_data[columns_summary["Growth Columns"]].values
    )
    upgrade_costs_df = pd.DataFrame(
        upgrade_costs,
        columns=[col.replace("median_price", "upgrade_costs") for col in aligned_columns["Median Price Columns"]],
        index=merged_data.index
    )
    merged_data_with_upgrades = pd.concat([merged_data, upgrade_costs_df], axis=1)
    print("\nUpgrade costs calculated and added successfully.")
else:
    print("\nNo alignment between growth and median price columns.")

# Inspect the result
merged_data_with_upgrades.head() if 'merged_data_with_upgrades' in locals() else merged_data.head()

Inspecting column patterns for alignment...

Median Price Columns: ['SizeRank_median_price', 'RegionType_median_price', '1/31/2023_median_price', '2/28/2023_median_price', '3/31/2023_median_price']...
Sold Above List Columns: ['SizeRank_sold_above_list', 'RegionType_sold_above_list', '1/31/2023_sold_above_list', '2/28/2023_sold_above_list', '3/31/2023_sold_above_list']...
Sold Below List Columns: ['SizeRank_sold_below_list', 'RegionType_sold_below_list', '1/31/2023_sold_below_list', '2/28/2023_sold_below_list', '3/31/2023_sold_below_list']...
Growth Columns: ['11/30/2024', '1/31/2025', '10/31/2025']

No alignment between growth and median price columns.


,RegionID,RegionName,StateName,SizeRank,RegionType,BaseDate,11/30/2024,1/31/2025,10/31/2025,SizeRank_value,...,12/31/2023_sold_below_list,1/31/2024_sold_below_list,2/29/2024_sold_below_list,3/31/2024_sold_below_list,4/30/2024_sold_below_list,5/31/2024_sold_below_list,6/30/2024_sold_below_list,7/31/2024_sold_below_list,8/31/2024_sold_below_list,9/30/2024_sold_below_list
0,394913,"New York, NY",NY,1,msa,10/31/2024,-0.5,-0.8,1.1,1,...,0.406383,0.428314,0.463802,0.419320,0.375195,0.357391,0.323827,0.331516,0.347761,0.378329
1,753899,"Los Angeles, CA",CA,2,msa,10/31/2024,-0.7,-0.8,2.2,2,...,0.426111,0.460034,0.383346,0.348691,0.325138,0.316547,0.333910,0.353801,0.394207,0.411886
2,394463,"Chicago, IL",IL,3,msa,10/31/2024,-1.0,-2.2,0.7,3,...,0.532863,0.543956,0.509206,0.450741,0.377788,0.366030,0.353733,0.368958,0.417902,0.459814
3,394514,"Dallas, TX",TX,4,msa,10/31/2024,-1.1,-2.1,1.3,4,...,0.613289,0.623790,0.583297,0.517837,0.511471,0.501287,0.535623,0.562001,0.608491,0.619420
4,394692,"Houston, TX",TX,5,msa,10/31/2024,-0.9,-1.7,0.8,5,...,0.676135,0.691283,0.647386,0.621079,0.593125,0.587870,0.590875,0.612032,0.645077,0.647457


In [9]:
# List columns to compare their formats
print("Growth Columns:", [col for col in merged_data.columns if col in ['11/30/2024', '1/31/2025', '10/31/2025']])
print("Median Price Columns:", [col for col in merged_data.columns if "median_price" in col])
print("Sold Above List Columns:", [col for col in merged_data.columns if "sold_above_list" in col])
print("Sold Below List Columns:", [col for col in merged_data.columns if "sold_below_list" in col])

Growth Columns: ['11/30/2024', '1/31/2025', '10/31/2025']
Median Price Columns: ['SizeRank_median_price', 'RegionType_median_price', '1/31/2023_median_price', '2/28/2023_median_price', '3/31/2023_median_price', '4/30/2023_median_price', '5/31/2023_median_price', '6/30/2023_median_price', '7/31/2023_median_price', '8/31/2023_median_price', '9/30/2023_median_price', '10/31/2023_median_price', '11/30/2023_median_price', '12/31/2023_median_price', '1/31/2024_median_price', '2/29/2024_median_price', '3/31/2024_median_price', '4/30/2024_median_price', '5/31/2024_median_price', '6/30/2024_median_price', '7/31/2024_median_price', '8/31/2024_median_price', '9/30/2024_median_price']
Sold Above List Columns: ['SizeRank_sold_above_list', 'RegionType_sold_above_list', '1/31/2023_sold_above_list', '2/28/2023_sold_above_list', '3/31/2023_sold_above_list', '4/30/2023_sold_above_list', '5/31/2023_sold_above_list', '6/30/2023_sold_above_list', '7/31/2023_sold_above_list', '8/31/2023_sold_above_list', '9

In [10]:
# Manually map growth dates to median price dates
column_mapping = {
    "11/30/2024": "10/31/2024_median_price",
    "1/31/2025": "12/31/2024_median_price",
    "10/31/2025": "9/30/2025_median_price"
}

# Use the mapping to align columns
growth_columns = list(column_mapping.keys())
price_columns = list(column_mapping.values())

In [11]:
# Calculate Upgrade Costs
upgrade_costs = merged_data[price_columns].values * merged_data[growth_columns].values

# Create a DataFrame for Upgrade Costs
upgrade_costs_df = pd.DataFrame(
    upgrade_costs,
    columns=[col.replace("median_price", "upgrade_costs") for col in price_columns],
    index=merged_data.index
)

# Merge with the original dataset
merged_data_with_upgrades = pd.concat([merged_data, upgrade_costs_df], axis=1)

# Inspect the updated dataset
merged_data_with_upgrades.head()

KeyError: "None of [Index(['10/31/2024_median_price', '12/31/2024_median_price',\n       '9/30/2025_median_price'],\n      dtype='object')] are in the [columns]"

In [12]:
# Print all column names in merged_data
print("All column names in the dataset:")
print(merged_data.columns.tolist())

All column names in the dataset:
['RegionID', 'RegionName', 'StateName', 'SizeRank', 'RegionType', 'BaseDate', '11/30/2024', '1/31/2025', '10/31/2025', 'SizeRank_value', 'RegionType_value', '1/31/2023', '2/28/2023', '3/31/2023', '4/30/2023', '5/31/2023', '6/30/2023', '7/31/2023', '8/31/2023', '9/30/2023', '10/31/2023', '11/30/2023', '12/31/2023', '1/31/2024', '2/29/2024', '3/31/2024', '4/30/2024', '5/31/2024', '6/30/2024', '7/31/2024', '8/31/2024', '9/30/2024', '10/31/2024', 'SizeRank_median_price', 'RegionType_median_price', '1/31/2023_median_price', '2/28/2023_median_price', '3/31/2023_median_price', '4/30/2023_median_price', '5/31/2023_median_price', '6/30/2023_median_price', '7/31/2023_median_price', '8/31/2023_median_price', '9/30/2023_median_price', '10/31/2023_median_price', '11/30/2023_median_price', '12/31/2023_median_price', '1/31/2024_median_price', '2/29/2024_median_price', '3/31/2024_median_price', '4/30/2024_median_price', '5/31/2024_median_price', '6/30/2024_median_price